# Student performance (Kaggle) (Moritz)
- https://www.kaggle.com/c/184702-tu-ml-ws-18-student-performance
- small samples (train = 198), medium dimension (32)
- attribute characteristics: numeric, categorical
- Predict: Grade
- Result file cols: id, Grade
- Missing values: No

## with preprocessing
- scale (fit train data to scaler, scale train and test data)
    - _SVR:_ very long runtime without scaling
- merge train and test data
- one hot encode (+ drop first columns) categorical data

### Linear Regression
R^2 and RMSE created from training data without(!) train-test split.
- without scaling 
- with all samples
    - <1 s
    - R^2: 0.40843
    - RMSE: 3.56123
    
    
- with preprocessing
- with all samples:
    - <1 s
    - R^2: 0.40843
    - RMSE: 3.56123
    - Kaggle: 4.75835
    
### SVR
- without scaling 
- with all samples
    - few seconds
    - C: 0.2, kernel: linear, epsilon: 0.5, gamma: auto 
    - RMSE: 4.25981
    - Kaggle: 4.47988
   
   
- with preprocessing
- with all samples:
    - few seconds
    - C: 0.2, kernel: linear, epsilon: 0.5, gamma: auto 
    - RMSE: 4.26364
    - Kaggle: 4.51673
    
### Gradient Boosted Decision Tree
- without scaling
- with all samples
    - 91.043 s
    - {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 150}
    - RMSE: 3.88092
    - Kaggle: 4.26606
    
    
- with preprocessing
- with all samples:
    - few minutes
    - {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 150}
    - RMSE: 3.8868
    - Kaggle: __4.24893__
    
### AutoML
- with preprocessing
- with all samples:
    - 

In [13]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
%run './base.ipynb'

In [14]:
# Import and preprocessing

# read train data
train = pd.read_csv('./data/student_performance_kaggle/StudentPerformance.shuf.train.csv')
# add index 'train' and val of id col
train['_index'] = 'train'
train.set_index(['_index', 'id'], inplace=True)
# extract, then drop 'Grade' col
train_target = train[['Grade']]
train.drop(['Grade'], axis='columns', inplace=True)

# read test data
test = pd.read_csv('./data/student_performance_kaggle/StudentPerformance.shuf.test.csv')
# add index 'test' and val of id coll
test['_index'] = 'test'
test.set_index(['_index', 'id'], inplace=True)

# scale train and test data
#train_s, test_s = scale_data(train, test)

# concat train and test data for futher preprocessing
#data_s = pd.concat([train_s, test_s])
data_s = pd.concat([train, test])

# one hot encode data
data_oh = one_hot(data_s, drop_first=True)

#display(data_oh)

# split data into train and test
X_train = data_oh.loc['train']
y_train = train_target
X_test = data_oh.loc['test']

display(X_train)

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
id,,,,,,,,,,,,,,,,,,,,,
70,15,3,1,2,4,0,4,4,2,2,...,0,0,0,1,0,0,0,1,1,0
275,17,2,4,2,2,0,4,3,3,1,...,1,0,0,1,1,0,1,1,1,1
94,16,4,2,2,2,0,5,3,3,1,...,1,0,0,1,1,1,1,1,1,0
276,17,2,2,2,2,0,4,4,4,2,...,1,0,1,1,1,0,1,1,1,1
241,17,4,3,2,2,0,2,5,5,1,...,1,0,0,0,0,1,1,1,1,1
339,18,3,3,1,4,0,5,3,3,1,...,1,0,0,1,0,0,1,1,1,0
226,18,3,1,1,2,1,5,3,3,1,...,1,0,0,0,0,1,1,1,1,1
273,18,1,1,2,2,0,4,4,3,1,...,1,0,0,1,1,0,0,1,0,0
98,16,2,1,1,2,0,4,3,5,1,...,1,0,0,1,1,0,1,1,0,1


In [9]:
# Linear Regression
reg = linear_reg(X_train, y_train, X_train, y_train)
result = pd.DataFrame(reg.predict(X_test), columns=['Grade'])

# join id col
result = pd.concat([X_test.reset_index()[['id']], result], axis='columns')

# Save result
filename = f'''lr_{dt.datetime.now()}.csv'''

result.to_csv('./predictions/student_performance_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

#display(result)

R^2 value for model: 0.40843
Predict:
RMSE: 3.56123
R^2 Score: 0.40843
Saved as lr_2019-01-05 13:03:22.158927.csv


In [12]:
# SVR
# params
param_grid = {
    'C': np.linspace(.2,1,5),
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], # poly very slow
    'epsilon': np.linspace(0,.5,6),
    'gamma': ['auto', 'scale']
}

# run grid search
gs = run_svr(X_train, y_train.values.ravel(), cv=5, param_grid=param_grid)

# predict
result = pd.DataFrame(gs.best_estimator_.predict(X_test), columns=['Grade'])

# join id col
result = pd.concat([X_test.reset_index()[['id']], result], axis='columns')
#display(result)

# Create SVR filename
filename = f'''svr_'''\
           f'''C-{gs.best_estimator_.C}_'''\
           f'''k-{gs.best_estimator_.kernel}_'''\
           f'''e-{gs.best_estimator_.epsilon}_'''\
           f'''g-{gs.best_estimator_.gamma}_'''\
           f'''{dt.datetime.now()}.csv'''

result.to_csv('./predictions/student_performance_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

GridSearch initializing...
SVR model in training...
MSE: 18.14595, RMSE: 4.25981, 
C: 0.2, kernel: linear, epsilon: 0.5, gamma: auto 
Saved as svr_C-0.2_k-linear_e-0.5_g-auto_2019-01-05 13:12:21.368568.csv


In [15]:
# Gradient Boosted Decision Tree
param_fix = {
    'learning_rate': .01, 
    'loss': 'ls'
}

param_grid = {
    'n_estimators': (50, 100, 150, 200, 300, 400, 500), 
    'max_depth': (1, 2, 3, 4, 5), 
    'min_samples_split': (2, 3, 5)
}

gs = run_boosted_tree(X_train, y_train.values.ravel(), [], [], param_fix=param_fix, cv=10, param_grid=param_grid)

#plot_scores(gbt.cv_results_)
#plot_training_deviance(gbt, test_data, test_target)

# predict
result = pd.DataFrame(gs.best_estimator_.predict(X_test), columns=['Grade'])

# join id col
result = pd.concat([X_test.reset_index()[['id']], result], axis='columns')
#display(result)

# Create SVR filename
filename = f'''gbdtree_'''\
           f'''ne-{gs.best_estimator_.n_estimators}_'''\
           f'''md-{gs.best_estimator_.max_depth}_'''\
           f'''mss-{gs.best_estimator_.min_samples_split}_'''\
           f'''{dt.datetime.now()}.csv'''

result.to_csv('./predictions/student_performance_kaggle/' + filename, sep = ",", index=False)
print(f'''Saved as {filename}''')

GridSearch initializing...
GradientBoostedRegressor model in training...
GradientBoostedRegressor model selected and fitted in 91.043 s

MSE: 15.06155, RMSE: 3.88092
Best parameters selected by GridSearch: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 150}
Saved as gbdtree_ne-150_md-5_mss-5_2019-01-05 13:16:29.721969.csv


In [11]:
# AutoML
run_autoML(X_train, y_train, X_train, y_train)


AutoML estimator initializing...
AutoML estimator in training...


KeyError: '[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17\n  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35\n  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53\n  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71\n  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89\n  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107\n 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125\n 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143\n 144 145 146 147 148 149 150 151 152 153 154 155 156 157] not in index'

In [137]:
# Unused
# Get a feeling for the dataset

# Check if train DataFrame has NaNs
if(train.isnull().values.any()): print('NaNs!')
else: print('Nons!')

Nons!
